# Lab 12 - Pivoting and unpivoting

This lab focuses on data transformation using pivoting and unpivoting operations with the polars library.

## Load your packages

You will need the `polars` package for this assignment. We'll also use `numpy` for some calculations.

In [1]:
# Import required packages
import polars as pl

# Question 1: Pivot US Economic Indicator Data

## 1a. Pivot data

Load the `economics_long.csv` dataset from the `data/` folder. You'll notice that in the `economics_long` dataset's `value` column, there are different types of measurements. Transform this dataset so that the unique values for the `variable` field become columns of their own.

**Please write code below to pivot the data. The output should be a DataFrame. You must "pivot" the data to receive full credit on this question.**

In [2]:
# Load economics_long data
economics_long = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/economics_long.csv")

economics_wide = (
    economics_long
    .with_columns(pl.col("date").str.strptime(pl.Date, strict=False))
    .pivot(values="value", index="date", columns="variable",
           aggregate_function="first", sort_columns=True)
    .sort("date")
)

# Display the pivoted dataframe
economics_wide


/tmp/ipython-input-1918390853.py:7: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  .pivot(values="value", index="date", columns="variable",


date,pce,pop,psavert,uempmed,unemploy
date,f64,f64,f64,f64,f64
1967-07-01,506.7,198712.0,12.6,4.5,2944.0
1967-08-01,509.8,198911.0,12.6,4.7,2945.0
1967-09-01,515.6,199113.0,11.9,4.6,2958.0
1967-10-01,512.2,199311.0,12.9,4.9,3143.0
1967-11-01,517.4,199498.0,12.8,4.7,3066.0
…,…,…,…,…,…
2014-12-01,12062.0,319746.157,7.6,12.9,8717.0
2015-01-01,12046.0,319928.646,7.7,13.2,8903.0
2015-02-01,12082.4,320074.511,7.9,12.9,8610.0


## 1b. Calculate median unemployment for 2010-01-01 and beyond

Next, compute the median unemployment metric (as defined by `unemploy`) for 2010-01-01 and beyond.

**Please write code below. The output should show the median value. You must use the dataframe created in 1a.**

In [3]:
median_unemploy = (
    economics_wide
    .filter(pl.col("date") >= pl.date(2010, 1, 1))
    .select(pl.col("unemploy").median().alias("median_unemploy"))
    .to_series()[0]
)

print(f"The median unemployment for 2010-01-01 and beyond is: {median_unemploy}")

The median unemployment for 2010-01-01 and beyond is: 12471.0


### Answer: The median unemployment for 2010-01-01 and beyond is: __________

12471.0

# Question 2: Air Passenger data

## 2a. Unpivot data

The `AirPassengers` dataset is wide, which makes it difficult to aggregate. Transform the `AirPassengers` dataset from wide to long. The resulting dataset should have three columns: one column representing year, one column representing month, and one column representing the number of air passengers.

**Please write code below to unpivot the data. The output should be a DataFrame. You must "unpivot" the data to receive full credit on this question.**

In [5]:
ir_passengers = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/AirPassengers.txt",
    separator="\t"
)

# Display the original data structure
print("Original data shape:", air_passengers.shape)
print("\nFirst few rows:")
print(air_passengers.head())


month_cols = [c for c in air_passengers.columns if c != "Year"]

air_passengers_long = (
    air_passengers
    .unpivot(
        on=month_cols,
        index=["Year"],
        variable_name="month",
        value_name="passengers"
    )
    .rename({"Year": "year"})
    .with_columns(pl.col("passengers").cast(pl.Int64))
    .sort(["year", "month"])
)

air_passengers_long

Original data shape: (12, 13)

First few rows:
shape: (5, 13)
┌──────┬─────┬─────┬─────┬───┬─────┬─────┬─────┬─────┐
│ Year ┆ Jan ┆ Feb ┆ Mar ┆ … ┆ Sep ┆ Oct ┆ Nov ┆ Dec │
│ ---  ┆ --- ┆ --- ┆ --- ┆   ┆ --- ┆ --- ┆ --- ┆ --- │
│ i64  ┆ i64 ┆ i64 ┆ i64 ┆   ┆ i64 ┆ i64 ┆ i64 ┆ i64 │
╞══════╪═════╪═════╪═════╪═══╪═════╪═════╪═════╪═════╡
│ 1949 ┆ 112 ┆ 118 ┆ 132 ┆ … ┆ 136 ┆ 119 ┆ 104 ┆ 118 │
│ 1950 ┆ 115 ┆ 126 ┆ 141 ┆ … ┆ 158 ┆ 133 ┆ 114 ┆ 140 │
│ 1951 ┆ 145 ┆ 150 ┆ 178 ┆ … ┆ 184 ┆ 162 ┆ 146 ┆ 166 │
│ 1952 ┆ 171 ┆ 180 ┆ 193 ┆ … ┆ 209 ┆ 191 ┆ 172 ┆ 194 │
│ 1953 ┆ 196 ┆ 196 ┆ 236 ┆ … ┆ 237 ┆ 211 ┆ 180 ┆ 201 │
└──────┴─────┴─────┴─────┴───┴─────┴─────┴─────┴─────┘


year,month,passengers
i64,str,i64
1949,"""Apr""",129
1949,"""Aug""",148
1949,"""Dec""",118
1949,"""Feb""",118
1949,"""Jan""",112
…,…,…
1960,"""Mar""",419
1960,"""May""",472
1960,"""Nov""",390


## 2b. Find the standard deviation between 1955 and 1960

Next, compute the standard deviation of passengers between (and including) the years 1955 and 1960.

**Please write code below. The output should show the standard deviation. You must use the dataframe created in 2a.**

In [6]:
std_passengers = (
    air_passengers_long
    .filter((pl.col("year") >= 1955) & (pl.col("year") <= 1960))
    .select(pl.col("passengers").std().alias("std_passengers"))
    .to_series()[0]
)

print(f"The standard deviation of airline passengers between 1955 and 1960 is: {std_passengers:.2f}")

The standard deviation of airline passengers between 1955 and 1960 is: 86.44


### The standard deviation of airline passengers between (and including) the years 1955 and 1960 is: __________

86.44

# Question 3: WHO tuberculosis case data

## 3a. Pivot data

Load the `table2.csv` dataset from the `data/` folder. This contains data for country populations and tuberculosis case metrics. You'll notice that the metrics for `cases` and `population` are in the same column (`count`). Pivot the data so that the metrics for `cases` and `population` are in their own columns.

**Please write code below to pivot the data. The output should be a DataFrame. You must "pivot" the data to receive full credit on this question.**

In [7]:
# Load table2 data
table2 = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/table2.csv")

# Display the original data structure
print("Original data:")
print(table2)

table2_wide = (
    table2
    .pivot(
        values="count",
        index=["country", "year"],
        columns="type",
        aggregate_function="first",   # safe for unique (country, year, type) rows
        sort_columns=True
    )
    .sort(["country", "year"])
)

table2_wide

Original data:
shape: (12, 5)
┌──────────┬─────────────┬──────┬────────────┬────────────┐
│ rownames ┆ country     ┆ year ┆ type       ┆ count      │
│ ---      ┆ ---         ┆ ---  ┆ ---        ┆ ---        │
│ i64      ┆ str         ┆ i64  ┆ str        ┆ i64        │
╞══════════╪═════════════╪══════╪════════════╪════════════╡
│ 1        ┆ Afghanistan ┆ 1999 ┆ cases      ┆ 745        │
│ 2        ┆ Afghanistan ┆ 1999 ┆ population ┆ 19987071   │
│ 3        ┆ Afghanistan ┆ 2000 ┆ cases      ┆ 2666       │
│ 4        ┆ Afghanistan ┆ 2000 ┆ population ┆ 20595360   │
│ 5        ┆ Brazil      ┆ 1999 ┆ cases      ┆ 37737      │
│ …        ┆ …           ┆ …    ┆ …          ┆ …          │
│ 8        ┆ Brazil      ┆ 2000 ┆ population ┆ 174504898  │
│ 9        ┆ China       ┆ 1999 ┆ cases      ┆ 212258     │
│ 10       ┆ China       ┆ 1999 ┆ population ┆ 1272915272 │
│ 11       ┆ China       ┆ 2000 ┆ cases      ┆ 213766     │
│ 12       ┆ China       ┆ 2000 ┆ population ┆ 1280428583 │
└─────────

/tmp/ipython-input-3740129727.py:10: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  .pivot(


country,year,cases,population
str,i64,i64,i64
"""Afghanistan""",1999,745,19987071
"""Afghanistan""",2000,2666,20595360
"""Brazil""",1999,37737,172006362
"""Brazil""",2000,80488,174504898
"""China""",1999,212258,1272915272
"""China""",2000,213766,1280428583


## 3b. Calculate highest cases/population ratio for 1999

Now that you have `cases` and `population` in their own columns, you can perform analysis. Create a new column in this dataset called `ratio` that divides `cases` by `population`. Next, filter the data to only include rows from the `year` of 1999. What is the country with the highest `cases`/`population` ratio?

**Please write code below. You must use the dataframe created in 3a.**

In [8]:
table2_with_ratio = table2_wide.with_columns(
    pl.when(pl.col("population") > 0)
      .then(pl.col("cases") / pl.col("population"))
      .otherwise(None)
      .alias("ratio")
)

# 2) filter for year 1999
year_1999 = table2_with_ratio.filter(pl.col("year") == 1999)

# 3) find country with highest ratio
highest_ratio_country = (
    year_1999
    .sort("ratio", descending=True)
    .select("country")
    .to_series()[0]
)

print("Data for 1999 with ratios:")
print(year_1999)
print(f"\nThe country with the highest cases/population ratio in 1999 is: {highest_ratio_country}")

Data for 1999 with ratios:
shape: (3, 5)
┌─────────────┬──────┬────────┬────────────┬──────────┐
│ country     ┆ year ┆ cases  ┆ population ┆ ratio    │
│ ---         ┆ ---  ┆ ---    ┆ ---        ┆ ---      │
│ str         ┆ i64  ┆ i64    ┆ i64        ┆ f64      │
╞═════════════╪══════╪════════╪════════════╪══════════╡
│ Afghanistan ┆ 1999 ┆ 745    ┆ 19987071   ┆ 0.000037 │
│ Brazil      ┆ 1999 ┆ 37737  ┆ 172006362  ┆ 0.000219 │
│ China       ┆ 1999 ┆ 212258 ┆ 1272915272 ┆ 0.000167 │
└─────────────┴──────┴────────┴────────────┴──────────┘

The country with the highest cases/population ratio in 1999 is: Brazil


### Answer: The country with the highest cases/population ratio in 1999 is: __________

## Submission Instructions

1. Complete all code cells above
2. Run all cells to ensure they execute without errors
3. Fill in the answer blanks with your computed values
4. Save and submit your completed notebook